<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 2

### Exploratory Data Analysis (EDA)

---

Your hometown mayor just created a new data analysis team to give policy advice, and the administration recruited _you_ via LinkedIn to join it. Unfortunately, due to budget constraints, for now the "team" is just you...

The mayor wants to start a new initiative to move the needle on one of two separate issues: high school education outcomes, or drug abuse in the community.

Also unfortunately, that is the entirety of what you've been told. And the mayor just went on a lobbyist-funded fact-finding trip in the Bahamas. In the meantime, you got your hands on two national datasets: one on SAT scores by state, and one on drug use by age. Start exploring these to look for useful patterns and possible hypotheses!

---

This project is focused on exploratory data analysis, aka "EDA". EDA is an essential part of the data science analysis pipeline. Failure to perform EDA before modeling is almost guaranteed to lead to bad models and faulty conclusions. What you do in this project are good practices for all projects going forward, especially those after this bootcamp!

This lab includes a variety of plotting problems. Much of the plotting code will be left up to you to find either in the lecture notes, or if not there, online. There are massive amounts of code snippets either in documentation or sites like [Stack Overflow](https://stackoverflow.com/search?q=%5Bpython%5D+seaborn) that have almost certainly done what you are trying to do.

**Get used to googling for code!** You will use it every single day as a data scientist, especially for visualization and plotting.

#### Package imports

In [ ]:
import numpy as np
import scipy.stats as stats
import csv
import pandas as pd

# this line tells jupyter notebook to put the plots in the notebook rather than saving them to file.
%matplotlib inline

# this line makes plots prettier on mac retina screens. If you don't have one it shouldn't do anything.
%config InlineBackend.figure_format = 'retina'

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Load the `sat_scores.csv` dataset and describe it

---

You should replace the placeholder path to the `sat_scores.csv` dataset below with your specific path to the file.

### 1.1 Load the file with the `csv` module and put it in a Python dictionary

The dictionary format for data will be the column names as key, and the data under each column as the values.

Toy example:
```python
data = {
    'column1':[0,1,2,3],
    'column2':['a','b','c','d']
    }
```

In [ ]:
sat_file = 'sat_scores.csv'
sat_dict = {}
with open(sat_file, 'r') as csv_file:
    # Using csv to read file
    sat_reader = csv.reader(csv_file)
    # To track if it's header or data rows
    row_count = 0
    for row in sat_reader:
        # Header row
        if row_count == 0:
            # Convert the header row into Dict keys
            sat_dict = sat_dict.fromkeys(row, [])
        # Data row
        else:
            for column, name in enumerate(sat_dict):
                # Add each element in a row to its associated keys in the Dict
                temp = sat_dict[name] + [row[column]]
                sat_dict[name] = temp
        row_count += 1
    csv_file.close()
sat_dict

In [ ]:
# Alternative approach to read csv
# raw_pd = ''
# sat_file = 'sat_scores.csv'
# with open(sat_file, 'r') as csvfile:
#     raw_pd = csvfile.read()
    
# raw_pd = raw_pd.splitlines()
# header = raw_pd[0].split(',')
# data = [data.split(',') for data in raw_pd[1:]]
# sat_dict = {head:[row[idx] for row in data] for idx, head in enumerate(header)}
# sat_dict

### 1.2 Make a pandas DataFrame object with the SAT dictionary, and another with the pandas `.read_csv()` function

Compare the DataFrames using the `.dtypes` attribute in the DataFrame objects. What is the difference between loading from file and inputting this dictionary (if any)?

In [ ]:
raw_to_pandas = pd.DataFrame(sat_dict)
print(raw_to_pandas.dtypes)
read_by_pandas = pd.read_csv(sat_file)
print(read_by_pandas.dtypes)

<div style='background-color:black;color:white;padding:5px'>
<h3 style='margin:3px'>Yes, the difference is that for the raw data, they are all read as string type, whereas Pandas will convert numeric values to integer types whenever possible.</h3>
</div>

If you did not convert the string column values to float in your dictionary, the columns in the DataFrame are of type `object` (which are string values, essentially). 

### 1.3 Look at the first ten rows of the DataFrame: what does our data describe?

From now on, use the DataFrame loaded from the file using the `.read_csv()` function.

Use the `.head(num)` built-in DataFrame function, where `num` is the number of rows to print out.

You are not given a "codebook" with this data, so you will have to make some (very minor) inference.

In [ ]:
read_by_pandas.head(10)

<div style='background-color:black;color:white;padding:5px;'>
<h3 style='margin:3px;color:white;'>
    'State': Probably all states in USA <br>
    'Rate': Participation rate by state <br>
    'Verbal': Average verbal score by state <br>
    'Math': Average math score by state
</h3>
</div>

In [ ]:
# Store and remove the last row of the data. It contains some information of all the states.
total_row = read_by_pandas.iloc[-1:]
read_by_pandas = read_by_pandas.iloc[:-1]
read_by_pandas.tail()

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Create a "data dictionary" based on the data

---

A data dictionary is an object that describes your data. This should contain the name of each variable (column), the type of the variable, your description of what the variable is, and the shape (rows and columns) of the entire dataset.

In [ ]:
data_dict = {column:{'type':read_by_pandas.dtypes[column]} for column in read_by_pandas.columns}
data_dict.update({'shape':read_by_pandas.shape})
data_dict['State'].update({'description':'Abbreviation of states in USA'})
data_dict['Rate'].update({'description':'Participation rate by state'})
data_dict['Math'].update({'description':'Average mathematics score by state'})
data_dict['Verbal'].update({'description':'Average verbal score by state'})
data_dict

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Plot the data using seaborn

---

### 3.1 Using seaborn's `distplot`, plot the distributions for each of `Rate`, `Math`, and `Verbal`

Set the keyword argument `kde=False`. This way you can actually see the counts within bins. You can adjust the number of bins to your liking. 

[Please read over the `distplot` documentation to learn about the arguments and fine-tune your chart if you want.](https://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.distplot.html#seaborn.distplot)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import seaborn as sns
sns.set(style = 'darkgrid')
# Plot the distribution for Rate
plt.figure(figsize=(15,5))
plot = sns.distplot(read_by_pandas['Rate'], kde=False, bins=8, color='red')
plot.set(ylabel='Frequency', title='Participation Rate')
for rect in plot.patches:
    plot.text(s = int(rect.get_height()), y = rect.get_height(), x = rect.get_x() + rect.get_width() / 2)
plt.show()
# Plot the distribution for Math
plt.figure(figsize=(15,5))
plot = sns.distplot(read_by_pandas['Math'], kde=False, bins=10, color='green', axlabel='Score');
plot.set(ylabel='Frequency', title='Math Score')
for rect in plot.patches:
    plot.text(s = int(rect.get_height()), y = rect.get_height(), x = rect.get_x() + rect.get_width() / 2)
plt.show()
# Plot the distribution for Verbal
plt.figure(figsize=(15,5))
plot = sns.distplot(read_by_pandas['Verbal'], kde=False, bins=10, color='orange', axlabel='Score');
plot.set(ylabel='Frequency', title='Verbal Score')
for rect in plot.patches:
    plot.text(s = int(rect.get_height()), y = rect.get_height(), x = rect.get_x() + rect.get_width() / 2)

### 3.2 Using seaborn's `pairplot`, show the joint distributions for each of `Rate`, `Math`, and `Verbal`

Explain what the visualization tells you about your data.

[Please read over the `pairplot` documentation to fine-tune your chart.](https://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.pairplot.html#seaborn.pairplot)

In [ ]:
sns.pairplot(read_by_pandas, height = 4, kind='reg');

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">Negative correlations are observed between "Rate" and "Math", and between "Rate" and "Verbal", whereas positive correlation is observed between "Math" and "Verbal". <br><br> The strength of the correlation between "Math" and "Verbal" appears stronger than the other two correlations.</h3>
</div>

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 4. Plot the data using built-in pandas functions.

---

Pandas is very powerful and contains a variety of nice, built-in plotting functions for your data. Read the documentation here to understand the capabilities:

http://pandas.pydata.org/pandas-docs/stable/visualization.html

### 4.1 Plot a stacked histogram with `Verbal` and `Math` using pandas

In [ ]:
ax = read_by_pandas[['Verbal', 'Math']].plot.hist(stacked=True, figsize=(15,5));
ax.set(xlabel = 'Score', title='Verbal and Math Scores (Stacked)');
for rect in ax.patches:
    if rect.get_height() > 0:
        ax.text(s = int(rect.get_height()), y = rect.get_y() + rect.get_height() - 1, x = rect.get_x() + rect.get_width() / 2)
plt.show();

### 4.2 Plot `Verbal` and `Math` on the same chart using boxplots

What are the benefits of using a boxplot as compared to a scatterplot or a histogram?

What's wrong with plotting a box-plot of `Rate` on the same chart as `Math` and `Verbal`?

In [ ]:
ax, bp = read_by_pandas[['Verbal', 'Math']].boxplot(return_type='both');
ax.set(ylabel = 'Score');
# [box.get_ydata() for box in bp["boxes"]]
# for i in bp:
#     for j in bp[i]:
#         print(j)

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">Benefits are a quick glance between multiple variables so that we can easily compared their mean, range, interquartile range (IQR) and outliers, if any.</h3>
</div>

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">Rate is a measure in percentage, ranged between 0 to 100, whereas Math and Verbal are absolute values with higher and wider ranges. Hence, by plotting a box-plot of Rate on the same chart as Math and Verbal, the comparsion will appear off and meaningless. </h3>
</div>

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 4.3 Plot `Verbal`, `Math`, and `Rate` appropriately on the same boxplot chart

Think about how you might change the variables so that they would make sense on the same chart. Explain your rationale for the choices on the chart. You should strive to make the chart as intuitive as possible. 


In [ ]:
read_by_pandas[['Rate','Math','Verbal']].apply(lambda x: (x-x.min())/(x.max()-x.min())).boxplot();

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">Using feature scaling like Max-Min normalization, all 3 variables will be subjected within the range of 0 and 1. The distance between each data points is now comparable and the values can also be used in machine learning algorithms. </h3>
</div>

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 5. Create and examine subsets of the data

---

For these questions you will practice **masking** in pandas. Masking uses conditional statements to select portions of your DataFrame (through boolean operations under the hood.)

Remember the distinction between DataFrame indexing functions in pandas:

    .iloc[row, col] : row and column are specified by index, which are integers
    .loc[row, col]  : row and column are specified by string "labels" (boolean arrays are allowed; useful for rows)
    .ix[row, col]   : row and column indexers can be a mix of labels and integer indices
    
For detailed reference and tutorial make sure to read over the pandas documentation:

http://pandas.pydata.org/pandas-docs/stable/indexing.html



### 5.1 Find the list of states that have `Verbal` scores greater than the average of `Verbal` scores across states

How many states are above the mean? What does this tell you about the distribution of `Verbal` scores?




In [ ]:
verbal_above_mean = read_by_pandas[read_by_pandas['Verbal'] > np.mean(read_by_pandas['Verbal'])]['State']
print('Number of states:', len(verbal_above_mean))
verbal_above_mean

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">This tells me that since there are 24 states above the mean (balance point) and 27 states below or equal to the mean, which both of them have about the same size, the distribution is likely to be symmetrical balance. </h3>
</div>

### 5.2 Find the list of states that have `Verbal` scores greater than the median of `Verbal` scores across states

How does this compare to the list of states greater than the mean of `Verbal` scores? Why?

In [ ]:
verbal_above_median = read_by_pandas[read_by_pandas['Verbal'] > np.median(read_by_pandas['Verbal'])]['State']
print('Number of states:', len(verbal_above_median))
verbal_above_median

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">The number of states that have verbal scores greater than the median of Verbal scores is equal to the number of states that are greater than the mean of Verbal scores. This reinforces the fact that the distribution is highly likely to be symmetrical. </h3>
</div>

### 5.3 Create a column that is the difference between the `Verbal` and `Math` scores

Specifically, this should be `Verbal - Math`.

In [ ]:
updated_sat_df = read_by_pandas.assign(Diff=(read_by_pandas['Verbal'] - read_by_pandas['Math']))
updated_sat_df.head()

### 5.4 Create two new DataFrames showing states with the greatest difference between scores

1. Your first DataFrame should be the 10 states with the greatest gap between `Verbal` and `Math` scores where `Verbal` is greater than `Math`. It should be sorted appropriately to show the ranking of states.
2. Your second DataFrame will be the inverse: states with the greatest gap between `Verbal` and `Math` such that `Math` is greater than `Verbal`. Again, this should be sorted appropriately to show rank.
3. Print the header of both variables, only showing the top 3 states in each.

In [ ]:
first_df = updated_sat_df.sort_values('Diff', ascending=False).head(10)
second_df = updated_sat_df.sort_values('Diff').head(10)
print(first_df.head(3))
print()
print(second_df.head(3))

## 6. Examine summary statistics

---

Checking the summary statistics for data is an essential step in the EDA process!

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 6.1 Create the correlation matrix of your variables (excluding `State`).

What does the correlation matrix tell you?


In [ ]:
read_by_pandas[['Rate','Math','Verbal']].corr()

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">Negative correlations are observed between "Rate" and "Math", and between "Rate" and "Verbal", whereas positive correlation is observed between "Math" and "Verbal". <br><br> The strength of the correlation between "Math" and "Verbal" is the strongest among the 3 correlations.</h3>
</div>

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 6.2 Use pandas'  `.describe()` built-in function on your DataFrame

Write up what each of the rows returned by the function indicate.

In [ ]:
read_by_pandas.describe()

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">
        count: number of data points <br>
        mean: average of each column <br>
        std: standard deviation of each column <br>
        min: minimum value of each column <br>
        25%:1st quartile (Q1) of each column <br>
        50%: median of each column <br>
        75%: 3rd quartile (Q3) of each column <br>
        max: maximum value of each column
    </h3>
</div>

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 6.3 Assign and print the _covariance_ matrix for the dataset

1. Describe how the covariance matrix is different from the correlation matrix.
2. What is the process to convert the covariance into the correlation?
3. Why is the correlation matrix preferred to the covariance matrix for examining relationships in your data?

In [ ]:
# Call the built-in function
read_by_pandas[['Rate','Math','Verbal']].cov()

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">
        The absolute values in a covariance matrix are much larger than that in a correlation matrix. Where the values in a correlation matrix are bounded between -1 and 1, the values in a covariance matrix are not bounded. 
    </h3>
</div>

In [ ]:
# What is the process to convert the covariance into the correlation?
cov_matrix = read_by_pandas[['Rate','Math','Verbal']].cov()
std_dev = read_by_pandas[['Rate','Math','Verbal']].std()
corr_matrix = cov_matrix.apply(lambda x: x/(std_dev[x.name]*std_dev[x.index]))
corr_matrix

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">
        To convert the covariance matrix into correlation matrix, simply divide the matrix by the standard deviations of the variables corresponding to the row index and the column index of the covariance matrix. <br>
        
\begin{equation}
Correlation = 
 \begin{pmatrix}
  \frac{Cov(1,1)}{\sigma(1) \times \sigma(1)} & \frac{Cov(1,2)}{\sigma(1) \times \sigma(2)} & \cdots & \frac{Cov(1,n)}{\sigma(1) \times \sigma(n)} \\
  \frac{Cov(2,1)}{\sigma(2) \times \sigma(1)} & \frac{Cov(2,2)}{\sigma(2) \times \sigma(2)} & \cdots & \frac{Cov(2,n)}{\sigma(2) \times \sigma(n)} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  \frac{Cov(n,1)}{\sigma(n) \times \sigma(1)} & \frac{Cov(n,2)}{\sigma(n) \times \sigma(2)} & \cdots & \frac{Cov(n,n)}{\sigma(n) \times \sigma(n)} 
 \end{pmatrix}
 \end{equation}
 
 \begin{equation}
Correlation = 
 \begin{pmatrix}
  1 & \frac{Cov(1,2)}{\sigma(1) \times \sigma(2)} & \cdots & \frac{Cov(1,n)}{\sigma(1) \times \sigma(n)} \\
  \frac{Cov(2,1)}{\sigma(2) \times \sigma(1)} & 1 & \cdots & \frac{Cov(2,n)}{\sigma(2) \times \sigma(n)} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  \frac{Cov(n,1)}{\sigma(n) \times \sigma(1)} & \frac{Cov(n,2)}{\sigma(n) \times \sigma(2)} & \cdots & 1 
 \end{pmatrix}
\end{equation}

   </h3>
</div>

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 7. Performing EDA on "drug use by age" data.

---

You will now switch datasets to one with many more variables. This section of the project is more open-ended - use the techniques you practiced above!

We'll work with the "drug-use-by-age.csv" data, sourced from and described here: https://github.com/fivethirtyeight/data/tree/master/drug-use-by-age.

### 7.1

Load the data using pandas. Does this data require cleaning? Are variables missing? How will this affect your approach to EDA on the data?

In [ ]:
drug_use_df = pd.read_csv('drug-use-by-age.csv')
drug_use_df.head()

In [ ]:
drug_use_df.info()

In [ ]:
drug_use_df.describe()

In [ ]:
for column in drug_use_df:
    print(drug_use_df[column].unique)

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">
Yes, this data requires cleaning. There are missing variables, and there is a need to change some values before EDA. For instance, alcohol-frequency is float type, but oxycontin-frequency is object type (likely string)
    </h3>
</div>

### 7.2 Do a high-level, initial overview of the data

Get a feel for what this dataset is all about.

Use whichever techniques you'd like, including those from the SAT dataset EDA. The final response to this question should be a written description of what you infer about the dataset.

Some things to consider doing:

- Look for relationships between variables and subsets of those variables' values
- Derive new features from the ones available to help your analysis
- Visualize everything!

In [ ]:
drug_use_w_nan = drug_use_df

In [ ]:
# Convert all columns, except age and n, into float type
# If not possible to convert, use NaN instead
def convert_float(value):
    """
    Attempt to convert inputs to float type, and if there is error converting, return NaN.
    Paramters:
    value: input value
    Return:
    float type of parameter value, or np.nan if conversion is impossible.
    """
    try:
        return float(value)
    except:
        return np.nan
drug_use_w_nan.iloc[:,2:] = drug_use_w_nan.iloc[:,2:].applymap(convert_float)

In [ ]:
# To check if the columns have been converted to float type, if possible. If not, convert it to NaN.
drug_use_w_nan.info()

In [ ]:
drug_use_w_nan.describe()

In [ ]:
frequency = [x for x in drug_use_w_nan.columns if 'frequency' in x]
use = [x for x in drug_use_w_nan.columns if 'use' in x]

In [ ]:
# Correlation between different drugs frequencies
plt.figure(figsize=(15,5))
sns.heatmap(drug_use_w_nan[frequency].corr(), center = 0, annot=True);

In [ ]:
# Correlation between different drugs uses
plt.figure(figsize=(15,5))
sns.heatmap(drug_use_w_nan[use].corr(), center = 0, annot=True);

In [ ]:
drug_use_1 = pd.get_dummies(drug_use_w_nan, columns=['age'])
drug_use_1.head()

In [ ]:
# Correlation between age group and drug frequencies, should someone in the age group abused the said drug
frequency_age = [x for x in drug_use_1.columns if 'frequency' in x or 'age' in x]
plt.figure(figsize=(15,5))
sns.heatmap(drug_use_1[frequency_age].corr(), center = 0);

In [ ]:
# Correlation between age group and drug use
use_age = [x for x in drug_use_1.columns if 'use' in x or 'age' in x]
plt.figure(figsize=(15,5))
sns.heatmap(drug_use_1[use_age].corr(), center=0);

In [ ]:
# finding the age band for which it has the highest frequency for each type of drug
drug_use_idx_age = drug_use_w_nan.set_index('age')
drug_use_idx_age[frequency].idxmax()

In [ ]:
# finding the age band for which it has the highest uses for each type of drug
drug_use_idx_age[use].idxmax()

<div style="background-color:#333;color:white;padding:10px; margin:10px;">
    <h3>My Inference on the Dataset</h3>
    <p> The first heatmap is the correlation between all the drug frequencies, should someone is currently abusing the drugs. </p>
    <p> It suggests that there are strong positive correlations between the frequency use of alcohol and marijuana, of cocaine and crack, of hallucinogen and inhalant, of stimulant and cocaine and of stimulant and crack. </p>
    <p> It also suggests that there are strong negative correlations between the frequency use of hallucinogen and marijuana, of inhalant and marijuana, of tranquilizer and alcohol and of tranquilizer and marijuana. </p><br>
    <p> The second heatmap is the correlation between all the drug uses. </p>
    <p> The most prominent result is that inhalant use has strong negative correlations, or at most weak positive correlations, with all the other drug use. </p>
    <p> Another result is that drug uses from pain releiver through meth has strong positive correlations among each other. </p><br>
    <p> The age band is further labelled using dummies variables and the result dataset is once again used to measure correlation between each drug frequencies/uses and each age band. 
    <p> Interestingly, the third heatmap (frequencies only) suggests that drug abusers above 65 has the strongest correlation to many drug frequencies. However, the fourth heatmap (uses only) suggest that there are strong negative correlations between participants above 65 years old and the proportion of them using different drugs.</p>
    <p>The fourth heatmap also suggest that for younger drug abusers, the correlation between them and the proportion of using inhalant drug is positively strongest among all drugs. This may suggest that inhalant may be much easier to be abused by younger drug abusers due to high accesibility and lower cost.</p>
    <p> The last result shows that participants below 30 has the highest proportion of drug abusers for each drug mentioned in the survey. </p>
</div>

### 7.3 Create a testable hypothesis about this data

Requirements for the question:

1. Write a specific question you would like to answer with the data (that can be accomplished with EDA).
2. Write a description of the "deliverables": what will you report after testing/examining your hypothesis?
3. Use EDA techniques of your choice, numeric and/or visual, to look into your question.
4. Write up your report on what you have found regarding the hypothesis about the data you came up with.


Your hypothesis could be on:

- Difference of group means
- Correlations between variables
- Anything else you think is interesting, testable, and meaningful!

**Important notes:**

You should be only doing EDA _relevant to your question_ here. It is easy to go down rabbit holes trying to look at every facet of your data, and so we want you to get in the practice of specifying a hypothesis you are interested in first and scoping your work to specifically answer that question.

Some of you may want to jump ahead to "modeling" data to answer your question. This is a topic addressed in the next project and **you should not do this for this project.** We specifically want you to not do modeling to emphasize the importance of performing EDA _before_ you jump to statistical analysis.

**Question and deliverables**

Question: Is there any association between age bands and type of drugs (alcohol and crack) used? 

Deliverables: p-value and determine if there is any statistically significance between the 2 categorical variables. 

In [ ]:
from scipy.stats import chi2_contingency
alcohol_num = (drug_use_w_nan['alcohol-use'] / 100 * drug_use_w_nan['n']).apply(int)
crack_num = (drug_use_w_nan['crack-use'] / 100 * drug_use_w_nan['n']).apply(int)
alcohol_crack_df = pd.DataFrame([alcohol_num, crack_num]).T
alcohol_crack_df.index = drug_use_w_nan['age']
alcohol_crack_df.columns = ['# of alcohol abusers', '# of crack abusers']
chi2, p, dof, expected = chi2_contingency(alcohol_crack_df)
print('p-value:', p)
alcohol_crack_df

In [ ]:
# Code
# from scipy.stats import kruskal
# import math
# median_list = [[i] for i in drug_use_w_nan['crack-frequency'] if math.isnan(i) == False]
# kruskal(*median_list)

**Report**

Since the p-value is less than 0.05, there is evidence showing it is statistically significant that there is an association between the age band and the choice of drug (alcohol and crack) abused. 

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 8. Introduction to dealing with outliers

---

Outliers are an interesting problem in statistics, in that there is not an agreed upon best way to define them. Subjectivity in selecting and analyzing data is a problem that will recur throughout the course.

1. Pull out the rate variable from the sat dataset.
2. Are there outliers in the dataset? Define, in words, how you _numerically define outliers._
3. Print out the outliers in the dataset.
4. Remove the outliers from the dataset.
5. Compare the mean, median, and standard deviation of the "cleaned" data without outliers to the original. What is different about them and why?

In [ ]:
original_rate = read_by_pandas[['Rate']]

### Define outliers as values less than (mean - 1.5 * standard deviation) or values more than (mean + 1.5 * standard deviation)

In [ ]:
original_mean = np.mean(original_rate['Rate'])
original_median = np.median(original_rate['Rate'])
original_std = np.std(original_rate['Rate'])
outliers = original_rate[(original_rate['Rate'] < original_mean - 1.5 * original_std) | (original_rate['Rate'] > original_mean + 1.5 * original_std)]
print(outliers)
cleaned_rate = original_rate.drop(outliers.index)
cleaned_mean = np.mean(cleaned_rate['Rate'])
cleaned_median = np.median(cleaned_rate['Rate'])
cleaned_std = np.std(cleaned_rate['Rate'])

In [ ]:
print('Original Mean:', original_mean)
print('Original Median:', original_median)
print('Original Standard Deviation:', original_std)
print()
print('Cleaned Mean:', cleaned_mean)
print('Cleaned Median:', cleaned_median)
print('Cleaned Standard Deviation:', cleaned_std)

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">
The mean, median and standard deviation have been reduced after removing the outliers. This is because given there are only 51 data points, which the dataset is relatively small, the outliers have the skew effect to pull the statistics towards their directions (positive direction). Hence, by removing them, this skew effect will be reduced. 
    </h3>
</div>

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 9. Percentile scoring and spearman rank correlation

---

### 9.1 Calculate the spearman correlation of sat `Verbal` and `Math`

1. How does the spearman correlation compare to the pearson correlation? 
2. Describe clearly in words the process of calculating the spearman rank correlation.
  - Hint: the word "rank" is in the name of the process for a reason!


In [ ]:
read_by_pandas[['Math','Verbal']].corr('pearson')

In [ ]:
read_by_pandas[['Math','Verbal']].corr('spearman')

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">
        1. Spearman correlation gives a higher value compared to pearson correlation.
    </h3>
</div>

<div style="background-color:#333;color:white;padding:10px;margin:10px;">
    <p> 1. Assign ranks to each value in each column (Math, Verbal), with rank 1 being the highest score in the respective column, followed by 2, 3, etc, and append the respective rank columns to each column (Math, Verbal). Sort the column first before assigning the rank number will be much easier! <p>
    <p> 2. Take the difference between the ranks of Math column and Verbal column and square it. The column shall be $d^2$. </p>
    <p> 3. Assume no tie in the rank columns, use this formula $$\rho = 1 - \frac{6\sum_{i} d_i^2}{n(n^2 - 1)}$$, where $n = \text{number of data points}$, to calculate the spearman rank correlation.
</div>

### 9.2 Percentile scoring

Look up percentile scoring of data. In other words, the conversion of numeric data to their equivalent percentile scores.

http://docs.scipy.org/doc/numpy-dev/reference/generated/numpy.percentile.html

http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html

1. Convert `Rate` to percentiles in the sat scores as a new column.
2. Show the percentile of California in `Rate`.
3. How is percentile related to the spearman rank correlation?

In [ ]:
new_column = pd.Series([stats.percentileofscore(read_by_pandas['Rate'], row) for row in read_by_pandas['Rate']])
convert_rate_sat = read_by_pandas.assign(Percentiles = new_column)
convert_rate_sat[convert_rate_sat['State']=='CA']['Percentiles']

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">
        3. Percentile rank also assign ranks to the values in a column, which the ranks can be used to calculate for spearman rank correlation. 
    </h3>
</div>

### 9.3 Percentiles and outliers

1. Why might percentile scoring be useful for dealing with outliers?
2. Plot the distribution of a variable of your choice from the drug use dataset.
3. Plot the same variable but percentile scored.
4. Describe the effect, visually, of coverting raw scores to percentile.

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">
        1. It is useful because outliers tend to skew the graph to a large extent. By using percentile scoring, this skewness will be greatly reduced when we only consider the distance between 2 percentiles and not the distance between their absolute values.
    </h3>
</div>

In [ ]:
print('2. Plot the distribution of a variable of your choice from the drug use dataset')
variable_choice = drug_use_df[['alcohol-use']]
plt.scatter(variable_choice.index, variable_choice);

In [ ]:
scored_variable = pd.DataFrame([stats.percentileofscore(variable_choice, row) for row in variable_choice.squeeze()])
print('3. Plot the same variable but percentile scored.')
plt.scatter(scored_variable.index, scored_variable);

<div style="background-color:#333;color:white;padding:10px">
    <h3 style="margin:10px">
4. After converting to percentile, the graph appears to be more linear prior to the conversion. The maximum value has been converted to 100.
    </h3>
</div>